In [23]:
suppressPackageStartupMessages(library(plyr))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
library(parallel)

# BR1

In [24]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V1/Dataset/scRNA_meta_data-2024-05-09.csv")
meta_data_subset<-meta_data %>% filter(Covid_exclusion=='no',
                                       sample.visitName%in% c('Flu Year 1 Day 0'))

In [25]:
df_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
df<-read.csv(paste0("/home/jupyter/BRI_Figures_Final_V1/Figure6/02_NMF_CD4_T_cell_Projection/NMF_Score_BRI/",x,".csv"))
df$pbmc_sample_id=x
return(df)
},mc.cores=60)

In [26]:
df_combined<-do.call(rbind,df_list)
df_combined_longformat<- pivot_longer(df_combined, cols = starts_with("NMF"), names_to = "NMF_Scores", values_to = "value")

### Get Mean NMF scores

In [27]:
df_combined_longformat <- df_combined_longformat %>%
  group_by(NMF_Scores, AIFI_L3) %>%
  summarise(
    AverageScore = mean(value),
    FractionNonZero = sum(value != 0) / n()
  )

`summarise()` has grouped output by 'NMF_Scores'. You can override using the
`.groups` argument.


In [29]:
df_combined_longformat_filtered<-df_combined_longformat %>% filter(NMF_Scores %in% c('NMF11_Th1','NMF5_Th2','NMF6_Tfh','NMF2_Th17')) %>% filter(AIFI_L3 %in% c('CM CD4 T cell','GZMB- CD27+ EM CD4 T cell',
                                                                            'GZMB- CD27- EM CD4 T cell','KLRF1- GZMB+ CD27- memory CD4 T cell'))

In [30]:
write.csv(df_combined_longformat_filtered,'NMF_Profile_Y1D0.csv')